In [ ]:
# ✅ 자동 분기 및 평가 통합 notebook 버전
import os, torch, numpy as np, csv, re
import torch.nn.functional as F
from datetime import datetime
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import resnet18, resnet34, densenet121, efficientnet_b0, efficientnet_v2_s
from glob import glob
import pandas as pd

# -------------------- 디바이스 및 경로 --------------------
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

slice_root = "/data1/lidc-idri/slices"
pth_root = "/home/iujeong/lung_cancer/pth"
log_path = "/home/iujeong/lung_cancer/logs/experiment_summary2.csv"
os.makedirs(os.path.dirname(log_path), exist_ok=True)

# -------------------- 라벨 추출 --------------------
def extract_label_from_filename(fname):
    score = int(fname.split("_")[-1].replace(".npy", ""))
    if score == 3:
        return None
    return int(score >= 4)

# -------------------- Dataset --------------------
class CTDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        path = self.paths[idx]
        label = self.labels[idx]
        img = np.load(path)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.
        img = np.expand_dims(img, axis=-1)
        if self.transform:
            img = self.transform(img)
        else:
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)
        return img, torch.tensor(label).long()

    def __len__(self):
        return len(self.paths)

# -------------------- Transform --------------------
test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# -------------------- Test Loader --------------------
all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)
from sklearn.model_selection import train_test_split
_, test_files, _, test_labels = train_test_split(files, labels, test_size=0.2, random_state=42)
test_loader = DataLoader(CTDataset(test_files, test_labels, transform=test_transform), batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

# -------------------- 클래스 수 자동 추출 --------------------
def get_output_classes_from_pth(pth_path):
    state_dict = torch.load(pth_path, map_location='cpu')
    # 우선 fc.weight → 보통 ResNet
    if 'fc.weight' in state_dict:
        return state_dict['fc.weight'].shape[0]
    # DenseNet
    elif 'classifier.weight' in state_dict:
        return state_dict['classifier.weight'].shape[0]
    # EfficientNet
    elif 'classifier.1.weight' in state_dict:
        return state_dict['classifier.1.weight'].shape[0]
    else:
        print(f"⚠️ 클래스 수 감지 실패 (기본값 2): {pth_path}")
        return 2

# -------------------- CBAM --------------------
class ChannelAttention(torch.nn.Module):
    def __init__(self, planes, ratio=16):
        super().__init__()
        self.shared = torch.nn.Sequential(
            torch.nn.Conv2d(planes, planes // ratio, 1, bias=False), torch.nn.ReLU(),
            torch.nn.Conv2d(planes // ratio, planes, 1, bias=False))
        self.avg, self.max, self.sigmoid = torch.nn.AdaptiveAvgPool2d(1), torch.nn.AdaptiveMaxPool2d(1), torch.nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.shared(self.avg(x)) + self.shared(self.max(x)))

class SpatialAttention(torch.nn.Module):
    def __init__(self, k=7):
        super().__init__()
        self.conv = torch.nn.Conv2d(2, 1, kernel_size=k, padding=k // 2, bias=False)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        avg = torch.mean(x, dim=1, keepdim=True)
        _max = torch.max(x, dim=1, keepdim=True)[0]
        return self.sigmoid(self.conv(torch.cat([avg, _max], dim=1)))

class CBAM(torch.nn.Module):
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()

    def forward(self, x):
        ca_out = self.ca(x) * x
        return self.sa(ca_out) * ca_out

class BasicBlockCBAM(torch.nn.Module):
    def __init__(self, in_planes, out_planes, stride=1, downsample=None, use_cbam=True):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(in_planes, out_planes, 3, stride, 1, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(out_planes)
        self.relu = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(out_planes, out_planes, 3, 1, 1, bias=False)
        self.bn2 = torch.nn.BatchNorm2d(out_planes)
        self.cbam = CBAM(out_planes) if use_cbam else None
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.cbam:
            out = self.cbam(out)
        if self.downsample:
            identity = self.downsample(x)
        return self.relu(out + identity)

class ResNet18_CBAM(torch.nn.Module):
    def __init__(self, num_classes=2, with_cbam=True):
        super().__init__()
        self.in_planes = 64
        self.with_cbam = with_cbam  # ✅ 여기 추가
        self.conv1 = torch.nn.Conv2d(1, 64, 7, 2, 3, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(64)
        self.relu = torch.nn.ReLU()
        self.maxpool = torch.nn.MaxPool2d(3, 2, 1)
        self.layer1 = self._make_layer(64, 2)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2, use_cbam=False)  # 마지막 블록은 CBAM 제외
        self.avgpool = torch.nn.AdaptiveAvgPool2d((1, 1))
        self.fc = torch.nn.Linear(512, num_classes)

    def _make_layer(self, planes, blocks, stride=1, use_cbam=None):
        if use_cbam is None:
            use_cbam = self.with_cbam  # ✅ 기본값 설정

        downsample = None
        if stride != 1 or self.in_planes != planes:
            downsample = torch.nn.Sequential(
                torch.nn.Conv2d(self.in_planes, planes, 1, stride, bias=False),
                torch.nn.BatchNorm2d(planes))
        layers = [BasicBlockCBAM(self.in_planes, planes, stride, downsample, use_cbam)]
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlockCBAM(planes, planes, use_cbam=use_cbam))
        return torch.nn.Sequential(*layers)

    def forward(self, x):
        x = self.maxpool(self.relu(self.bn1(self.conv1(x))))
        x = self.layer4(self.layer3(self.layer2(self.layer1(x))))
        x = self.avgpool(x)
        return self.fc(torch.flatten(x, 1))

# -------------------- 모델 타입 자동 감지 --------------------
def detect_model_type_safe(pth_path):
    try:
        state_dict = torch.load(pth_path, map_location='cpu')
        keys = set(state_dict.keys())
        if any('cbam' in k for k in keys):
            return "resnet18_cbam"
        elif any(k.startswith("classifier") for k in keys):
            if 'features.denseblock3' in ''.join(keys):
                return "densenet121"
            elif 'features.6' in ''.join(keys):
                return "effnetb0"
            elif 'features.7' in ''.join(keys):
                return "effnetv2s"
        elif any(k.startswith("layer4") for k in keys) and any(k.startswith("fc") for k in keys):
            if state_dict['fc.weight'].shape[1] == 512:
                return "resnet18"
            elif state_dict['fc.weight'].shape[1] == 512 and 'layer3.5.conv1.weight' in keys:
                return "resnet34"
    except Exception as e:
        print(f"[!] {pth_path} 감지 실패: {e}")
    return "unknown"

# -------------------- 전체 감지 및 저장 --------------------
def detect_model_type_safe(pth_path):
    try:
        state_dict = torch.load(pth_path, map_location='cpu')
        keys = set(state_dict.keys())

        if any('cbam' in k for k in keys):
            return "resnet18_cbam"
        elif 'classifier.1.weight' in keys:
            if 'features.6.0.weight' in keys:
                return "effnetb0"
            elif 'features.7.0.weight' in keys:
                return "effnetv2s"
        elif 'features.denseblock3.denselayer16.conv2.weight' in keys:
            return "densenet121"
        elif 'layer4.1.conv1.weight' in keys:
            if 'layer3.5.conv1.weight' in keys:
                return "resnet34"
            else:
                return "resnet18"
    except Exception as e:
        print(f"[!] {pth_path} 감지 실패: {e}")
    return "unknown"

def detect_all_models(pth_root, save_csv="detected_models.csv"):
    pths = [f for f in os.listdir(pth_root) if f.endswith(".pth")]
    results = []
    for p in pths:
        full_path = os.path.join(pth_root, p)
        model_type = detect_model_type_safe(full_path)
        results.append({"pth_path": p, "detected_model": model_type})
    df = pd.DataFrame(results)
    df.to_csv(save_csv, index=False)
    print(f"✅ 모델 감지 결과 저장됨: {save_csv}")

# ----
def model_loader_fn(model_name, version="plain", pth_path=None):
    num_classes = get_output_classes_from_pth(pth_path) if pth_path else 2

    if version == "plain":
        if model_name == "resnet18":
            model = resnet18(weights=None)
            model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
            model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

        elif model_name == "resnet34":
            model = resnet34(weights=None)
            model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
            model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

        elif model_name == "densenet121":
            num_classes = get_output_classes_from_pth(pth_path)  # 이게 1로 나오면
            model = densenet121(num_classes=num_classes)  # 이렇게 맞춰서 로드
            model.features.conv0 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
            model.classifier = torch.nn.Linear(model.classifier.in_features, num_classes)

        elif model_name == "effnetb0":
            model = efficientnet_b0(weights=None)
            model.features[0][0] = torch.nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)
            model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)

        elif model_name == "effnetv2s":
            model = efficientnet_v2_s(weights=None)
            model.features[0][0] = torch.nn.Conv2d(1, 24, kernel_size=3, stride=2, padding=1, bias=False)
            model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)

        else:
            raise ValueError(f"[!] Unknown plain model: {model_name}")

        return model, num_classes

    elif version in ["cbam", "cbam_mga"]:
        model = ResNet18_CBAM(num_classes=num_classes, with_cbam=True)
        return model, num_classes

    else:
        raise ValueError(f"[!] Unknown model version: {version}")


# -------------------- 로그 저장 --------------------
def save_to_csv(log_path, exp_dict, results):
    exists = os.path.exists(log_path)
    with open(log_path, 'a', newline='') as f:
        writer = csv.writer(f)
        if not exists:
            writer.writerow(["timestamp", "model", "version", "augmentation", "cbam", "mga", "loss", "scheduler", "accuracy", "auc", "f1", "sensitivity", "specificity", "pth_file"])
        writer.writerow([
            datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            exp_dict['model'], exp_dict['version'], exp_dict['augmentation'], exp_dict['cbam'], exp_dict['mga'],
            exp_dict['loss'], exp_dict['scheduler'],
            *["%.4f" % v for v in results], exp_dict['pth_path']
        ])

# ✅ 여기에 실험 리스트 넣고 run_all_experiments 호출하면 끝!

experiment_list = pd.DataFrame([
    {"model": "resnet18", "pth_path": "resnet18_baseline.pth", "augmentation": "Resize224", "cbam": False, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "resnet18_best_aug1.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": False, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "resnet18_best_aug2.pth", "augmentation": "CenterCrop224+Flip+Rotate+Blur", "cbam": False, "mga": False, "loss": "CEL", "scheduler": "None"},
    
    {"model": "resnet18", "pth_path": "aug_resnet_cbam1.pth", "augmentation": "CenterCrop180+Flip+Rotate+Blur", "cbam": True, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam.pth", "augmentation": "RandomResizedCrop224+Flip+Rotate", "cbam": True, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam_1.pth", "augmentation": "Resize224+Rotate+Erase", "cbam": True, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam_3.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": True, "mga": True, "loss": "CEL+MGA", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam_4.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": True, "mga": True, "loss": "CEL+MGA", "scheduler": "Lambda"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam_8.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": True, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam_9.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": True, "mga": False, "loss": "CEL", "scheduler": "StepLR"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam_focalloss.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": True, "mga": False, "loss": "FocalLoss", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam_smoothing.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": True, "mga": False, "loss": "LabelSmoothing", "scheduler": "None"},

    {"model": "resnet34", "pth_path": "best_model_resnet34.pth", "augmentation": "Resize224", "cbam": False, "mga": False, "loss": "CEL", "scheduler": "None"},

    {"model": "densenet121", "pth_path": "best_model_densenet121.pth", "augmentation": "Resize224", "cbam": False, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "densenet121", "pth_path": "aug_resnet_cbam_mga.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": False, "mga": True, "loss": "CEL+MGA", "scheduler": "None"},

    {"model": "effnetb0", "pth_path": "best_model_efficientnet_b0.pth", "augmentation": "Resize224", "cbam": False, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "effnetv2s", "pth_path": "best_model_effnetv2s.pth", "augmentation": "Resize224", "cbam": False, "mga": False, "loss": "CEL", "scheduler": "None"}
])

# ✅ 버전 자동 추가
experiment_list["version"] = experiment_list.apply(
    lambda row: "cbam_mga" if row["cbam"] and row["mga"]
    else "cbam" if row["cbam"]
    else "plain", axis=1
)
# ✅ 이후 run_all_experiments 함수에서 model_loader_fn(exp["model"], exp["version"]) 으로 호출



# -------------------- 평가 및 저장 --------------------
def evaluate_model(model, test_loader, num_classes):
    model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            probs = F.softmax(outputs, dim=1)[:, 1] if num_classes > 1 else torch.sigmoid(outputs).squeeze()
            preds = outputs.argmax(1) if num_classes > 1 else (probs > 0.5).long()
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    acc = (np.array(y_pred) == np.array(y_true)).mean()
    auc = roc_auc_score(y_true, y_probs)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel() if cm.shape == (2,2) else (0,0,0,0)
    sens = tp / (tp + fn + 1e-6)
    spec = tn / (tn + fp + 1e-6)
    return acc, auc, f1, sens, spec

def save_to_csv(log_path, exp_dict, results):
    exists = os.path.exists(log_path)
    with open(log_path, 'a', newline='') as f:
        writer = csv.writer(f)
        if not exists:
            writer.writerow(["timestamp", "model", "version", "augmentation", "cbam", "mga", "loss", "scheduler", "accuracy", "auc", "f1", "sensitivity", "specificity", "pth_file"])
        writer.writerow([datetime.now().strftime("%Y-%m-%d %H:%M:%S"), exp_dict['model'], exp_dict['version'], exp_dict['augmentation'], exp_dict['cbam'], exp_dict['mga'], exp_dict['loss'], exp_dict['scheduler'], *[f"{v:.4f}" for v in results], exp_dict['pth_path']])

# ✅ 실행 예시
# experiment_list = pd.read_csv("/home/iujeong/lung_cancer/logs/experiment_config.csv")
experiment_list["version"] = experiment_list.apply(lambda row: "cbam" if row["cbam"] else "plain", axis=1)

def run_one_class_experiments(model_list):
    for fname in model_list:
        path = os.path.join(pth_root, fname)
        model_type = detect_model_type_safe(path)
        print(f"▶ 1-class Evaluating: {fname} ({model_type})")
        try:
            model, _ = model_loader_fn(model_type.replace("resnet18_cbam", "resnet18"), "plain", path)
            model.load_state_dict(torch.load(path, map_location=device), strict=False)
            model = model.to(device)

            results = evaluate_model(model, test_loader, num_classes=1)
            exp_dict = {
                'model': model_type,
                'version': 'plain',
                'augmentation': 'unknown',
                'cbam': False,
                'mga': False,
                'loss': 'unknown',
                'scheduler': 'unknown',
                'pth_path': fname
            }
            save_to_csv(log_path, exp_dict, results)
            print(f"✅ 저장 완료 (1-class): {fname}\n")
        except Exception as e:
            print(f"❌ 오류 발생 ({fname}): {e}")

if input("▶ 평가 시작할까요? (y/n): ").lower().startswith("y"):
    run_all_experiments(experiment_list, test_loader)

Using device: cuda:1


In [58]:
import os, csv, torch, numpy as np
import pandas as pd
from datetime import datetime
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score
import torch.nn.functional as F
from torchvision.models import resnet18, resnet34, densenet121, efficientnet_b0, efficientnet_v2_s
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

# 디바이스 및 경로 설정
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
pth_root = "/home/iujeong/lung_cancer/pth"
log_path = "/home/iujeong/lung_cancer/logs/oneclass_summary.csv"
os.makedirs(os.path.dirname(log_path), exist_ok=True)

# 클래스 수 감지
def get_class_count(pth_path):
    state_dict = torch.load(pth_path, map_location='cpu')
    if 'fc.weight' in state_dict:
        return state_dict['fc.weight'].shape[0]
    elif 'classifier.weight' in state_dict:
        return state_dict['classifier.weight'].shape[0]
    return None

# 모델 타입 감지
def detect_model_type_safe(pth_path):
    state_dict = torch.load(pth_path, map_location='cpu')
    keys = set(state_dict.keys())
    if any("denseblock" in k for k in keys):
        return "densenet121"
    elif any("features.6" in k for k in keys):
        return "effnetb0"
    elif any("features.7" in k for k in keys):
        return "effnetv2s"
    elif "fc.weight" in state_dict:
        if "layer3.5.conv1.weight" in keys:
            return "resnet34"
        return "resnet18"
    return "unknown"

# 모델 로딩 함수 (1-class용)
def model_loader_fn(model_name):
    if model_name == "resnet18":
        model = resnet18(weights=None)
        model.conv1 = torch.nn.Conv2d(1, 64, 7, 2, 3, bias=False)
        model.fc = torch.nn.Linear(model.fc.in_features, 1)
    elif model_name == "resnet34":
        model = resnet34(weights=None)
        model.conv1 = torch.nn.Conv2d(1, 64, 7, 2, 3, bias=False)
        model.fc = torch.nn.Linear(model.fc.in_features, 1)
    elif model_name == "densenet121":
        model = densenet121(num_classes=1)
        model.features.conv0 = torch.nn.Conv2d(1, 64, 7, 2, 3, bias=False)
        model.classifier = torch.nn.Linear(model.classifier.in_features, 1)
    elif model_name == "effnetb0":
        model = efficientnet_b0(weights=None)
        model.features[0][0] = torch.nn.Conv2d(1, 32, 3, 2, 1, bias=False)
        model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 1)
    elif model_name == "effnetv2s":
        model = efficientnet_v2_s(weights=None)
        model.features[0][0] = torch.nn.Conv2d(1, 24, 3, 2, 1, bias=False)
        model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 1)
    else:
        raise ValueError(f"Unknown model type: {model_name}")
    return model

# ✅ 여기에 실험 리스트 넣고 run_all_experiments 호출하면 끝!

experiment_list = pd.DataFrame([
    {"model": "resnet18", "pth_path": "resnet18_baseline.pth", "augmentation": "Resize224", "cbam": False, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "resnet18_best_aug1.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": False, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "resnet18_best_aug2.pth", "augmentation": "CenterCrop224+Flip+Rotate+Blur", "cbam": False, "mga": False, "loss": "CEL", "scheduler": "None"},
    
    {"model": "resnet18", "pth_path": "aug_resnet_cbam1.pth", "augmentation": "CenterCrop180+Flip+Rotate+Blur", "cbam": True, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam.pth", "augmentation": "RandomResizedCrop224+Flip+Rotate", "cbam": True, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam_1.pth", "augmentation": "Resize224+Rotate+Erase", "cbam": True, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam_3.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": True, "mga": True, "loss": "CEL+MGA", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam_4.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": True, "mga": True, "loss": "CEL+MGA", "scheduler": "Lambda"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam_8.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": True, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam_9.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": True, "mga": False, "loss": "CEL", "scheduler": "StepLR"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam_focalloss.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": True, "mga": False, "loss": "FocalLoss", "scheduler": "None"},
    {"model": "resnet18", "pth_path": "aug_resnet_cbam_smoothing.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": True, "mga": False, "loss": "LabelSmoothing", "scheduler": "None"},

    {"model": "resnet34", "pth_path": "best_model_resnet34.pth", "augmentation": "Resize224", "cbam": False, "mga": False, "loss": "CEL", "scheduler": "None"},

    {"model": "densenet121", "pth_path": "best_model_densenet121.pth", "augmentation": "Resize224", "cbam": False, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "densenet121", "pth_path": "aug_resnet_cbam_mga.pth", "augmentation": "Resize224+Flip+Rotate+Erase", "cbam": False, "mga": True, "loss": "CEL+MGA", "scheduler": "None"},

    {"model": "effnetb0", "pth_path": "best_model_efficientnet_b0.pth", "augmentation": "Resize224", "cbam": False, "mga": False, "loss": "CEL", "scheduler": "None"},
    {"model": "effnetv2s", "pth_path": "best_model_effnetv2s.pth", "augmentation": "Resize224", "cbam": False, "mga": False, "loss": "CEL", "scheduler": "None"}
])

# ✅ 버전 자동 추가
experiment_list["version"] = experiment_list.apply(
    lambda row: "cbam_mga" if row["cbam"] and row["mga"]
    else "cbam" if row["cbam"]
    else "plain", axis=1
)
# ✅ 이후 run_all_experiments 함수에서 model_loader_fn(exp["model"], exp["version"]) 으로 호출





# 평가 함수
def evaluate_model(model, test_loader):
    model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            probs = torch.sigmoid(outputs).squeeze()
            preds = (probs > 0.5).long()
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    acc = (np.array(y_pred) == np.array(y_true)).mean()
    auc = roc_auc_score(y_true, y_probs)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
    sens = tp / (tp + fn + 1e-6)
    spec = tn / (tn + fp + 1e-6)
    return acc, auc, f1, sens, spec

# 로그 저장
def save_to_csv(log_path, model_name, pth_file, results):
    exists = os.path.exists(log_path)
    with open(log_path, 'a', newline='') as f:
        writer = csv.writer(f)
        if not exists:
            writer.writerow(["timestamp", "model", "accuracy", "auc", "f1", "sensitivity", "specificity", "pth_file"])
        writer.writerow([
            datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            model_name, *["%.4f" % r for r in results], pth_file
        ])

# 평가용 Dummy TestLoader
class DummyDataset(Dataset):
    def __getitem__(self, idx):
        img = np.random.rand(224, 224).astype(np.float32)
        img = (img - 0.5) / 0.5
        img = np.expand_dims(img, axis=0)
        return torch.tensor(img), torch.tensor(1)
    def __len__(self): return 100

test_loader = DataLoader(DummyDataset(), batch_size=16)

# 전체 실행 루프
def run_oneclass_evaluation():
    all_pths = sorted([f for f in os.listdir(pth_root) if f.endswith(".pth")])
    for fname in all_pths:
        full_path = os.path.join(pth_root, fname)
        class_count = get_class_count(full_path)
        if class_count != 1:
            print(f"⚠️ Skip (not 1-class): {fname}")
            continue
        try:
            model_type = detect_model_type_safe(full_path)
            model = model_loader_fn(model_type)
            model.load_state_dict(torch.load(full_path, map_location=device), strict=False)
            model.to(device)
            results = evaluate_model(model, test_loader)
            save_to_csv(log_path, model_type, fname, results)
            print(f"✅ 저장 완료: {fname}")
        except Exception as e:
            print(f"❌ 오류 발생: {fname} → {e}")

# ✅ 실행
run_oneclass_evaluation()

/opt/anaconda3/envs/lungcancer/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/opt/anaconda3/envs/lungcancer/lib/python3.12/site-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


✅ 저장 완료: aug_densenet121_1.pth
⚠️ Skip (not 1-class): aug_resnet_cbam.pth
⚠️ Skip (not 1-class): aug_resnet_cbam1.pth
⚠️ Skip (not 1-class): aug_resnet_cbam_1.pth
⚠️ Skip (not 1-class): aug_resnet_cbam_3.pth
⚠️ Skip (not 1-class): aug_resnet_cbam_4.pth
⚠️ Skip (not 1-class): aug_resnet_cbam_5.pth
⚠️ Skip (not 1-class): aug_resnet_cbam_6.pth
⚠️ Skip (not 1-class): aug_resnet_cbam_8.pth
⚠️ Skip (not 1-class): aug_resnet_cbam_9.pth
⚠️ Skip (not 1-class): aug_resnet_cbam_focalloss.pth
⚠️ Skip (not 1-class): aug_resnet_cbam_mga.pth
⚠️ Skip (not 1-class): aug_resnet_cbam_smoothing.pth


/opt/anaconda3/envs/lungcancer/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


✅ 저장 완료: best_aug_densenet121_baseline.pth


/opt/anaconda3/envs/lungcancer/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/opt/anaconda3/envs/lungcancer/lib/python3.12/site-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


...

/opt/anaconda3/envs/lungcancer/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


✅ 저장 완료: best_model_resnet34.pth
❌ 오류 발생: best_model_resnet34_3ch.pth → Error(s) in loading state_dict for ResNet:
	size mismatch for conv1.weight: copying a param with shape torch.Size([64, 3, 7, 7]) from checkpoint, the shape in current model is torch.Size([64, 1, 7, 7]).


/opt/anaconda3/envs/lungcancer/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


✅ 저장 완료: best_model_resnet34_pretatin.pth
✅ 저장 완료: model_aug_resnet18.pth
⚠️ Skip (not 1-class): resnet18_baseline.pth
⚠️ Skip (not 1-class): resnet18_baseline_aug1.pth
⚠️ Skip (not 1-class): resnet18_best_aug1.pth
⚠️ Skip (not 1-class): resnet18_best_aug2.pth
⚠️ Skip (not 1-class): resnet18_cbam_mga.pth


/opt/anaconda3/envs/lungcancer/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, roc_auc_score

# 기본 지표
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
auc = roc_auc_score(y_true, y_probs)

# 특이도 계산
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
specificity = tn / (tn + fp)

# 출력
print(f"\n📊 Final Test Metrics from .pth:")
print(f"✅ Accuracy   : {accuracy:.4f}")
print(f"✅ Precision  : {precision:.4f}")
print(f"✅ Recall     : {recall:.4f} (Sensitivity)")
print(f"✅ Specificity: {specificity:.4f}")
print(f"✅ F1-score   : {f1:.4f}")
print(f"✅ AUC        : {auc:.4f}")
print("✅ Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))